In [ ]:
#Import required packages
import cv2 as cv
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2 as cv
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import pandas as pd
from Helper_functions import load_dict
from AI_functions import resnet18,CellDataset_supervised,data_generator
import torch.nn as nn
import torch.optim as optim


In [ ]:
#Insert filepath for local files  FOR THIBAUT
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\BoneMarrow_sample1"
readpath = basepath + "\\Raw_Images"
outpath = basepath + "\\Outputs"
file_prefix="\\sample1_"
maskpath=basepath+"\\ExportedMasks"
image_dim=64 #Dim of the final images
nuclear_channel="Ch7"
cellmask_channel="Ch1_mask"
df=pd.read_csv(outpath+"\\cell_info.csv")
cell_names=df["Cell_ID"].to_numpy()


In [ ]:
image_dict=load_dict(outpath,cell_names,image_dim)

In [ ]:
Channels=['Ch1']  #Channel to be fed to the NN
images_with_index = []
for image_i in image_dict:
    image=cv.merge([image_dict[image_i][i] for i in Channels])
    images_with_index.append((int(image_i),image))
    
images=np.array([image[1] for image in images_with_index])
names=np.array([image[0] for image in images_with_index])
assert sum(names!=df['Cell_ID'].to_numpy()) ==0  #Check that the order has been preserved
DNA_pos=df["DNA_pos"].to_numpy()
Touches_Boundary=df["Touches_boundary"].to_numpy()

labels=df[["Intensity_MC_Ch02",	"Intensity_MC_Ch03"	,"Intensity_MC_Ch11"]].to_numpy()

Thresh=50
# plt.hist(df["Gradient RMS_M01_Ch01"],bins=200);
# plt.axvline(x=Thresh,color="red")
# plt.show()
idx_to_keep=np.array(DNA_pos==1,dtype=int)+np.array(Touches_Boundary==0,dtype=int)+np.array(df["Gradient RMS_M01_Ch01"]>Thresh,dtype=int)==3 #keep dnapos, no touch boundarym APC and Other
#Filter
images=images[idx_to_keep]
names=names[idx_to_keep]
labels=labels[idx_to_keep]


mini=int(round(abs(np.array(images).min()),0))
images=images+abs(np.array(images).min())
mean=np.array(images).mean()
maxi=np.array(images).max()
std=np.array(images).std()

In [ ]:
#Scale and normalize the labels for regression
from Helper_functions import log_pol_scale
labels=log_pol_scale(labels)

# plt.hist(labels[:,0],bins=100);
# plt.show()
# plt.hist(labels[:,1],bins=100);
# plt.show()
# plt.hist(labels[:,2],bins=100);
# print(np.mean(labels,axis=0))

In [ ]:
del image_dict

In [ ]:
images,labels,names=images[:1000],labels[:1000],names[:1000]

In [ ]:
returned=data_generator(images,labels,names,mini,train_test_split = 0.8,batch_size = 100,sample=False)
train,test,batch_size,mean_loader,std_loader=returned
[train_data,train_data1,train_labels,train_ID]=train
[test_data,test_data1,test_labels,test_ID]=test

In [ ]:
transform_train = transforms.Compose(
    [transforms.ToPILImage(),transforms.RandomHorizontalFlip(p=0.5),transforms.RandomVerticalFlip(p=0.5),transforms.RandomRotation(degrees=180,fill=mini),
    transforms.ToTensor(),
    transforms.Normalize(mean_loader,std_loader)  
])

transform_validation = transforms.Compose(
    [transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean_loader,std_loader)  
])

In [ ]:
class CellDataset_supervised():
    def __init__(self, images,labels,ID, transforms=None):
        self.X = images
        self.Y=  labels
        self.Z= ID
        self.transforms = transforms
         
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        data = self.X[i]
        label=self.Y[i]
        ID=self.Z[i]
        data = np.asarray(data).astype(np.float32)
        label = np.asarray(label).astype(np.float32)
        
        if self.transforms:
            data = self.transforms(data)
        
        return data,label,ID

In [ ]:
train_data = CellDataset_supervised(train_data1,train_labels,train_ID, transform_train)
test_data = CellDataset_supervised(test_data1,test_labels, test_ID,transform_validation)


#Create DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
np.shape(train_labels)

In [ ]:
def train_epoch(NN, device, dataloader, loss_fn, optimizer,noise_factor=0):
    # Set train mode for both the encoder and the decoder
    NN.train()
    train_loss = []
    total=0
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    for image_batch,labels_batch,_ in dataloader: # with "_" we just ignore the labels (the second element of the dataloader tuple)
        image_noisy = image_batch
        image_batch = image_noisy.to(device)
        labels_batch=labels_batch.to(device)
        output = NN(image_batch)
        # Evaluate loss
        loss = loss_fn(output,labels_batch)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.detach().cpu().numpy())
        total+=labels_batch.size(0)
        output=output.detach().cpu().numpy()
        labels=labels_batch.detach().cpu().numpy()
    return np.mean(train_loss)

### validationing function
def validation_epoch(NN, device, dataloader, loss_fn):
    NN.eval()
    total=0
    val_loss=[]
    with torch.no_grad(): # No need to track the gradients
        for image_batch,labels_batch,ID_batch in dataloader:
            # Move tensor to the proper device
            image_batch = image_batch.to(device)
            labels_batch = labels_batch.to(device)
            output = NN(image_batch)
            loss = loss_fn(output,labels_batch)
            total+=labels_batch.size(0)
            val_loss.append(loss.detach().cpu().numpy())

    return np.mean(val_loss)


In [ ]:
Channels

In [ ]:
num_epochs=1
lr=0.001
num_classes=3

ConvNet_simple=resnet18(channel_num=len(Channels),num_classes=num_classes)
device = "cuda" if torch.cuda.is_available() else "cpu"
ConvNet_simple.to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(ConvNet_simple.parameters(), lr = lr) 

diz_loss = {'train_loss':[],'val_loss':[]}

for epoch in range(num_epochs):

    train_loss = train_epoch(ConvNet_simple,device,train_loader,loss_fn,optimizer)
    val_loss = validation_epoch(ConvNet_simple,device,validation_loader,loss_fn)
    
    print('\n EPOCH',epoch+1,' \t train loss',train_loss,' \t val loss',val_loss)
    diz_loss['train_loss'].append(train_loss)
    diz_loss['val_loss'].append(val_loss)


#_ = loss_over_epochs(diz_loss,num_epochs)

In [ ]:
test_data = CellDataset_supervised(test_data1,test_labels,test_ID)
test_loader = DataLoader(test_data, batch_size=124, shuffle=True)#,sampler=sampler)

predictions=np.zeros((len(test_data1),num_classes))
names=np.zeros((len(test_data1)))
labels=np.zeros((len(test_data1),num_classes))
i=0
ConvNet_simple.eval()
with torch.no_grad():
    for X, Y,ID in test_loader:
        X = X.to(device)
        latents=ConvNet_simple(X[:,None]).detach().cpu().numpy()
        labels[i:i+len(latents)]=Y
        names[i:i+len(latents)]=ID
        predictions[i:i+len(latents)]=latents
        i+=len(latents)


In [ ]:
df["Cell_ID"]=pd.to_numeric(df['Cell_ID'])
df10=pd.DataFrame()
df10['Cell_ID']=names
df10["PRED_Intensity_MC_Ch02"]=predictions[:,0]
df10["PRED_Intensity_MC_Ch03"]=predictions[:,1]
df10["PRED_Intensity_MC_Ch11"]=predictions[:,2]
df10["SCALED_Intensity_MC_Ch02"]=labels[:,0]
df10["SCALED_Intensity_MC_Ch03"]=labels[:,1]
df10["SCALED_Intensity_MC_Ch11"]=labels[:,2]
df10.to_csv(basepath+"/Results/Resnet"+str(diz_loss['train_loss'][-1])+".csv", index = False, header=True)
